In [1]:
import $file.common, common._
import $file.quel, quel._
import cats.data._, cats._, cats.implicits._
import doobie.implicits._
import doobie.util.fragment._
import doobie.util._, query._
import cats.evidence.Is

import $file.$     , common._

import $file.$   , quel._

import cats.data._, cats._, cats.implicits._

import doobie.implicits._

import doobie.util.fragment._

import doobie.util._, query._

import cats.evidence.Is

# Variation 6b. From QUEΛ to SQL 

The main purpose of this notebook is showing how can we generate efficient SQL queries from QUEΛ programs. Before that, however, we will do a simpler exercise to demonstrate the power of tagless-final APIs.

### A pretty-printer of QUEΛ programs

The question is this: can we actually write a pretty printer for the `largeCapitals` MTL-based query in [variance 4](Variance4.MTL.ipynb)?

`def largeCapitals[F[_]: Monad: FunctorFilter](implicit W: WorldRepo[F]): F[(String, String)]`

The answer is we can't. This pretty printer would require instantiating the corresponding APIs with a constant type constructor like `F[T] = String`. This could be certainly be done for the `WorldRepo` API, but not for `Monad` and `FunctorFilter`. Indeed, how could we implement the following signatures?

In [2]:
type ConstString[T] = String 

object StringMonad extends Monad[ConstString]{
    def flatMap[A, B](fa: String)(f: A => String): String = ???
    def pure[A](a: A): String = ???
    def tailRecM[A, B](a: A)(f: A => String): String = ???
}

object StringFunctorFilter extends FunctorFilter[ConstString]{
    def functor = ???
    def mapFilter[A, B](fa: String)(f: A => Option[B]): String = ???
}

defined type ConstString
defined object StringMonad
defined object StringFunctorFilter

The only thing we could implement in these instantiations is the `functor` field of `FunctorFilter`. The other components, we can't - not even the `pure[A]` component of `Monad` (we would need a `Show[A]` instance). For example, in the case of `flatMap`, how are we suppose to obtain a pretty-print representation of the continuation `A => String`? This Scala function is a black-box which we can't inspect. We may try to obtain a representation of the function body, but we need an argument of type `A` to start with. And how are we supposed to obtain that value? This should be _computed_ from `fa: String`, but `String` is not a computation at all. What we are looking for is a _representation_ of that continuation, and tagless-final APIs fit the bill! 

Let's give a non-standard semantics of QUEΛ for pretty-printing:

In [3]:
abstract class StringRep[T] extends (String => Int => String)

object StringRep{
    
    def apply[T](f: String => Int => String): StringRep[T] = 
        new StringRep[T]{
            def apply(i: String): Int => String = f(i)
        }

    implicit object StringQUEΛ extends QUEΛ[StringRep]{
        // Base types

        def bool(b: Boolean): StringRep[Boolean] = 
            StringRep(_ => _ => s"Q.bool(${b.toString})")

        def int(i: Int): StringRep[Int] = 
            StringRep(_ => _ => s"Q.int(${i.toString})")

        def str(s: String): StringRep[String] = 
            StringRep(_ => _ => s"Q.str($s)")
                
        def ===(a1: StringRep[Int], a2: StringRep[Int]): StringRep[Boolean] = 
            StringRep(t => i => s"Q.===(${a1(t)(i)}, ${a2(t)(i)})")

        def >(i1: StringRep[Int], i2: StringRep[Int]): StringRep[Boolean] = 
            StringRep(t => i => s"Q.>(${i1(t)(i)}, ${i2(t)(i)})")

        def &&(i1: StringRep[Boolean], i2: StringRep[Boolean]): StringRep[Boolean] = 
            StringRep(t => i => s"Q.&&(${i1(t)(i)}, ${i2(t)(i)})")

        // ADTs

        def tuple2[A, B](a: StringRep[A], b: StringRep[B]): StringRep[(A, B)] = 
            StringRep(t => i => s"""Q.tuple2(${a(t)(i)}, ${b(t)(i)})""")

        def none[A]: StringRep[Option[A]] = 
            StringRep(_ => _ => "none")

        def some[A](a: StringRep[A]): StringRep[Option[A]] = 
            StringRep(t => i => s"some(${a(t)(i)})")

        def exists[A](o: StringRep[Option[A]])(cond: StringRep[A] => StringRep[Boolean]): StringRep[Boolean] = 
            StringRep(t => i => s"""Q.exists(${o(t)(i)})(x$i => ${cond(_ => _ => s"x$i")(t)(i)})""")

        // Comprehensions

        def from[A, B](q: StringRep[List[A]])(f: StringRep[A] => StringRep[List[B]]): StringRep[List[B]] = 
            StringRep(t => i => 
                s"""|${t}Q.from(
                    |${q(t+"    ")(i)}
                    |${t}){ x$i => 
                    |${f(_ => _ => s"x$i")(t+"    ")(i+1)}
                    |${t}}""".stripMargin)

        def select[A](a: StringRep[A]): StringRep[List[A]] = 
            StringRep(t => i => t + s"Q.select(${a(t)(i)})")

        def where[A](cond: StringRep[Boolean])(q: StringRep[List[A]]): StringRep[List[A]] = 
            StringRep(t => i => 
                s"""|${t}Q.where(${cond(t)(i)})(
                    |${q(t+"    ")(i)}
                    |${t})""".stripMargin)
    }
    
    implicit object StringRepWorldModel extends WorldModel[StringRep]{
        // Cities
        
        def cityId(city: StringRep[City]): StringRep[Int] = 
            StringRep(t => i => s"W.cityId(${city(t)(i)})")

        def cityName(city: StringRep[City]): StringRep[String] = 
            StringRep(t => i => s"W.cityName(${city(t)(i)})")

        def cityCountry(city: StringRep[City]): StringRep[String] = 
            StringRep(t => i => s"W.cityCountry(${city(t)(i)})")

        def cityPopulation(city: StringRep[City]): StringRep[Int] = 
            StringRep(t => i => s"W.cityPopulation(${city(t)(i)})")

        // Countries
        
        def countryCode(country: StringRep[Country]): StringRep[String] = 
            StringRep(t => i => s"W.countryCode(${country(t)(i)})")

        def countryName(country: StringRep[Country]): StringRep[String] = 
            StringRep(t => i => s"W.countryName(${country(t)(i)})")

        def countryCapital(country: StringRep[Country]): StringRep[Option[Int]] = 
            StringRep(t => i => s"W.countryCapital(${country(t)(i)})")

        // World
        
        def allCountries: StringRep[List[Country]] = 
            StringRep(t => _ => t + s"W.allCountries")

        def allCities: StringRep[List[City]] = 
            StringRep(t => _ => t + s"W.allCities")
    }
}

defined class StringRep
defined object StringRep

The first argument in the representation type `String => Int => String` takes into account tabbing, and the second one is a counter used for giving names to variables (which appear in `exist` and `from`). Note that `from` is certainly similar to `flatMap`, how is it that we could implement it? Let's pay attention to the `from` signature:

`def from[A, B](q: Repr[List[A]])(f: Repr[A] => Repr[List[B]]): Repr[List[B]]`

As we can see, the continuation's type does not demand an `A` but a `Repr[A]`, i.e. we don't need a computation of `A`, but just a representation, which we can certainly provide. 

Let's obtain the pretty print representation of our `largeCapitals` query: 

In [4]:
import QUEΛSyntax._, WorldModelSyntax._

def largeCapitals[Repr[_]: QUEΛ: WorldModel]: Repr[List[(String, String)]] = 
    for {
        country <- allCountries
        city <- allCities if country.capital.exists(_ === city.id)
        if city.population > 8000000
    } yield (city.name, country.name)

import QUEΛSyntax._, WorldModelSyntax._


defined function largeCapitals

In [5]:
println(largeCapitals[StringRep].apply("    ")(0))

    Q.from(
        W.allCountries
    ){ x0 => 
        Q.from(
            Q.from(
                Q.from(
                    W.allCities
                ){ x1 => 
                    Q.where(Q.exists(W.countryCapital(x0))(x2 => Q.===(x2, W.cityId(x1))))(
                        Q.select(x1)
                    )
                }
            ){ x1 => 
                Q.where(Q.>(W.cityPopulation(x1), Q.int(8000000)))(
                    Q.select(x1)
                )
            }
        ){ x1 => 
            Q.select(Q.tuple2(W.cityName(x1), W.countryName(x0)))
        }
    }


Note that there is no trace of the for-comprehension syntax, since the Scala compiler desugars these expressions into corresponding `flatMap`, `filter` and `map` calls. However, we can't see these invocations either, where are they? They were also desugared, this time by our interpreter. In fact, tagless-final interpreters behave like pre-processors, and derived functions like `flatMap`, `map` and `filter` implemented in the `QUEΛSyntax` module, are actually much like macros!

Last, note that we obtain valid Scala code, as we can manually check by copy-pasting the resulting `String` in the following signature:

In [6]:
def largeCapitalsPP[Repr[_]](implicit Q: QUEΛ[Repr], W: WorldModel[Repr]): Repr[List[(String, String)]] = 
    Q.from(
        W.allCountries
    ){ x0 => 
        Q.from(
            Q.from(
                Q.from(
                    W.allCities
                ){ x1 => 
                    Q.where(Q.exists(W.countryCapital(x0))(x2 => Q.===(x2, W.cityId(x1))))(
                        Q.select(x1)
                    )
                }
            ){ x1 => 
                Q.where(Q.>(W.cityPopulation(x1), Q.int(8000000)))(
                    Q.select(x1)
                )
            }
        ){ x1 => 
            Q.select(Q.tuple2(W.cityName(x1), W.countryName(x0)))
        }
    }

defined function largeCapitalsPP

In [7]:
largeCapitalsPP[StringRep].apply("")(0) == largeCapitals[StringRep].apply("")(0)

res6: Boolean = true

### Show me the SQL query!

We are ready now to face the real challenge: generate SQL expressions from `QUEΛ` queries. Informally, there seems to be a close correspondence between comprehensions and SQL: loosely speaking, `from` expressions generates a whole SQL statement; `where` expressions contribute to the SQL `WHERE` clause; and `select` expressions to the SQL `SELECT` one. Where do basic tables for the SQL `FROM` clause come from? They correspond to domain model expressions of type `List[A]`, for some type `A`. In the world domain model these are defined by `allCountries` and `allCities`. Ok, but if we analyse the pretty print result of `largeCapitals` we stumbled upon a mess of nested `from` expressions, that would certainly lead to nested subqueries, if at all correct. Right, let's look to the following _normalized_ query instead, in order to see the correspondence of `QUEΛ` to `SQL` more directly:

In [8]:
def largeCapitalsNormalized[Repr[_]](implicit Q: QUEΛ[Repr], W: WorldModel[Repr]): Repr[List[(String, String)]] = 
    Q.from(W.allCountries){ x0 => 
        Q.from(W.allCities){ x1 => 
            Q.where(W.countryCapital(x0).exists(_ === W.cityId(x1)))(
                Q.where(W.cityPopulation(x1) > Q.int(8000000))(
                    Q.select((W.cityName(x1), W.countryName(x0)))
                )
            )
        }
    }

defined function largeCapitalsNormalized

In [9]:
println(largeCapitalsNormalized[StringRep].apply("    ")(0))

    Q.from(
        W.allCountries
    ){ x0 => 
        Q.from(
            W.allCities
        ){ x1 => 
            Q.where(Q.exists(W.countryCapital(x0))(x2 => Q.===(x2, W.cityId(x1))))(
                Q.where(Q.>(W.cityPopulation(x1), Q.int(8000000)))(
                    Q.select(Q.tuple2(W.cityName(x1), W.countryName(x0)))
                )
            )
        }
    }


This query is said to be in _normal form_: very informally, we can understand it as a linearly ordered sequence of `from` expressions, followed by `where` expressions, and ended by the `select` one. This can be easily translated to SQL. But the problem is, then, how to transform any arbitrary, possibly messy, `QUEΛ` expression into its normal form. There are two major alternatives: we may implement a re-writing interpreter that performs syntactic transformations until the normal form is found; or we may perform the bulk of the normalization process at the semantic level, coming back to the syntactic level to generate the normal form as a simple step. This second strategy is called _normalization by evaluation_ and its application to our problem is explained in the following [paper](http://okmij.org/ftp/meta-programming/#SQUR) by Oleg and Tatsuya:

![](images/maintainingpaper.png)

In what follows, we will apply the normalization-by-evaluation strategy introduced in this paper. The implementation is carried out in terms of a [tagless-final "optimization" interpreter](http://okmij.org/ftp/tagless-final/course2/index.html), which allows us to generate the normalized expression in _one shot_. This is the representation of the interpreter, which in essence follows the OCaml [implementation](http://okmij.org/ftp/meta-programming/Sqr/) of the paper:

In [10]:
sealed abstract class Normalized[Repr[_], T]
case class Unk[Repr[_], A](rep: Repr[A]) extends Normalized[Repr, A]
case class Table[Repr[_], A](rep: Repr[List[A]]) extends Normalized[Repr, List[A]]
case class Compr[Repr[_], A, B](table: Table[Repr, A], 
                                cont: Repr[A] => Normalized[Repr, List[B]]) 
    extends Normalized[Repr, List[B]]
case class Single[Repr[_], A](cond: List[Repr[Boolean]], returns: Repr[A]) 
    extends Normalized[Repr, List[A]]

object Normalized{
    
    def inj[Repr[_], A](rep: Repr[A]): Normalized[Repr, A] = 
        Unk(rep)
    
    def dyn[Repr[_]: QUEΛ, A](expr: Normalized[Repr, A]): Repr[A] = expr match {
        case Unk(rep) => rep
        case Table(rep) => rep
        case Single(Nil, ret) => 
            QUEΛ[Repr].select(ret)
        case Single(conds, ret) => 
            QUEΛ[Repr].where(conds.reduce(QUEΛ[Repr].&&))(
                QUEΛ[Repr].select(ret))
        case d: Compr[_, _, _] => 
            QUEΛ[Repr].from(d.table.rep)(d.cont andThen dyn[Repr, A])
    }
    
    implicit class Dyn[Repr[_]: QUEΛ, A](expr: Normalized[Repr, A]){
        def dyn: Repr[A] = Normalized.dyn(expr)
    }
}

implicit class Inj[Repr[_], A](rep: Repr[A]){
    def inj: Normalized[Repr, A] = Unk(rep)
}

defined class Normalized
defined class Unk
defined class Table
defined class Compr
defined class Single
defined object Normalized
defined class Inj

In [11]:
import QUEΛSyntax._

import QUEΛSyntax._

Wow, the `Id[_]` and `StringRep[_]` representations were more simple! Let's not despair, this is certainly more complex, but the complexity will hopefully vanish if we note that the semantic domain is a GADT that represents a _normalized_ expression, whose cases correspond to different _classes_ of QUEΛ expressions (of a generic, arbitrary representation type `Repr`) we are naturally interested in, given our normalization purpose: 

* `Table`: These are QUEΛ expressions that represent raw _tables_, such as `allCountries` and `allCities`. These are the heads in normalized comprehensions expressions.
* `Compr`: A normalized comprehension expression starting with a table, and followed by a normalized expression. 
* `Single`: A single expression consisting of a where clause with a number of conditions, and a single selected result. Normalized comprehension expressions are typically finished with this type of expression (although they may also end up with a table).
* `Unk`: QUEΛ expressions we are not interested in giving a normal form for them: basically, expressions of base types and ADTs.

The functions `inj` and `dyn` are characteristic of tagless-final optimizing interpreters: the former allow us to convert an already normalized expression in the `Normalized` representation; the latter allows us to obtain the normalized expression in the base representation `Repr`.

Let's check out now how the normalization process proceeds:

In [12]:
class NormalizedQUEΛ[Repr[_]: QUEΛ] extends QUEΛ[Normalized[Repr, ?]]{
    val U = QUEΛ[Repr]
    
    // base types

    def bool(b: Boolean): Normalized[Repr, Boolean] = 
        U.bool(b).inj

    def int(i: Int): Normalized[Repr, Int] = 
        U.int(i).inj

    def str(s: String): Normalized[Repr, String] = 
        U.str(s).inj

    def ===(a1: Normalized[Repr, Int], a2: Normalized[Repr, Int]): Normalized[Repr, Boolean] = 
        U.===(a1.dyn, a2.dyn).inj

    def >(i1: Normalized[Repr, Int], i2: Normalized[Repr, Int]): Normalized[Repr, Boolean] = 
        U.>(i1.dyn, i2.dyn).inj
    
    def &&(i1: Normalized[Repr, Boolean], i2: Normalized[Repr, Boolean]): Normalized[Repr, Boolean] = 
        U.&&(i1.dyn, i2.dyn).inj
    
    // ADTs

    def tuple2[A, B](a: Normalized[Repr, A], b: Normalized[Repr, B]): Normalized[Repr, (A, B)] = 
        U.tuple2(a.dyn, b.dyn).inj

    def none[A]: Normalized[Repr, Option[A]] = 
        U.none.inj

    def some[A](a: Normalized[Repr, A]): Normalized[Repr, Option[A]] = 
        U.some(a.dyn).inj

    def exists[A](o: Normalized[Repr, Option[A]])(cond: Normalized[Repr, A] => Normalized[Repr, Boolean]): Normalized[Repr, Boolean] =
        U.exists(o.dyn)(repa => cond(repa.inj).dyn).inj

    // Comprehensions

    def from[A, B](q: Normalized[Repr, List[A]])(cont: Normalized[Repr, A] => Normalized[Repr, List[B]]): Normalized[Repr, List[B]] =
        q match {
            case Unk(q) => from(Table(q))(cont)
            case t@Table(rep) => Compr(t, (Normalized.inj[Repr,A] _) andThen cont)
            case Single(Nil, ret) => cont(ret.inj)
            case Single(conds, ret) => where(conds.reduce(U.&&).inj)(cont(ret.inj))
            case c: Compr[Repr, _, _] => fromCompr(c, cont)
        }

    private def fromCompr[A, B, C](compr: Compr[Repr, C, A], 
            cont: Normalized[Repr, A] => Normalized[Repr, List[B]]): Normalized[Repr, List[B]] = 
        Compr(compr.table, (repc: Repr[C]) => from(compr.cont(repc))(cont))

    def where[A](cond: Normalized[Repr, Boolean])(q: Normalized[Repr, List[A]]): Normalized[Repr, List[A]] = 
        q match {
            case Unk(q) => where(cond)(Table(q))
            case t@Table(rep) => Compr(t, (repa: Repr[A]) => where(cond)(select(repa.inj)))
            case Single(conds2, ret) => Single(cond.dyn :: conds2, ret)
            case Compr(table, contBA) => Compr(table, contBA andThen (where(cond)(_)))
        }
    
    def select[A](a: Normalized[Repr, A]): Normalized[Repr, List[A]] = 
        Single(Nil, a.dyn)
}

defined class NormalizedQUEΛ

If you are not familiar to the tagless-final style, but you are to Free monads, what we are doing here is akin to the left-bind re-association process performed [here](https://github.com/typelevel/cats/blob/master/free/src/main/scala/cats/free/Free.scala#L49). What it's done at a syntactic level in the initial style, i.e. GADTs, it's done semantically in the tagless-final style. More concretely, it's done in a compositional way: the different syntactic constructs receive normalized expressions as arguments, and its interpretation proceeds to compose these basic components to create the normalized expression of the construct. For instance, a comprehension expression `from` receives an already normalized expression and a continuation. In order to create the normalized comprehension expression, we inspect which type of normalized expression we have received: if it's a single unconditional expression, we simply apply the continuation to return whatever normalized expression it gives us; if it's another comprehension expression, then we re-associate "left-binds" in `fromCompr`. Besides this flattening of `from` expressions, we also fuse `where` expressions, so that we will end up with a single `where` clause. 

This is the bulk of the normalization process, but note that the `largeCapital` query also builds upon the `WorldModel` API. So, we also need to provide normalized expressions for these basic domain constructs:

In [13]:
class NormalizedWorldModel[Repr[_]: WorldModel: QUEΛ] extends WorldModel[Normalized[Repr, ?]]{
    val U = implicitly[WorldModel[Repr]]
    
    // Cities

    def cityId(city: Normalized[Repr, City]): Normalized[Repr, Int] = 
        U.cityId(city.dyn).inj 

    def cityName(city: Normalized[Repr, City]): Normalized[Repr, String] = 
        U.cityName(city.dyn).inj 

    def cityCountry(city: Normalized[Repr, City]): Normalized[Repr, String] = 
        U.cityCountry(city.dyn).inj 

    def cityPopulation(city: Normalized[Repr, City]): Normalized[Repr, Int] = 
        U.cityPopulation(city.dyn).inj 

    // Countries

    def countryName(country: Normalized[Repr, Country]): Normalized[Repr, String] = 
        U.countryName(country.dyn).inj 
    
    def countryCapital(country: Normalized[Repr, Country]): Normalized[Repr, Option[Int]] = 
        U.countryCapital(country.dyn).inj 
    
    // World

    def allCountries: Normalized[Repr, List[Country]] =
        Table(U.allCountries)

    def allCities: Normalized[Repr, List[City]] = 
        Table(U.allCities)
}


defined class NormalizedWorldModel

As we can see, most expressions are already normalized. The only interesting part is the semantics of `allCountries` and `allCities`: these expressions are _annotated_ as basic `Table`s, so that they can play the role of heads in normalized comprehension expressions.

In [14]:
largeCapitalsNormalized[StringRep].apply("")(0)

res13: String = """Q.from(
    W.allCountries
){ x0 => 
    Q.from(
        W.allCities
    ){ x1 => 
        Q.where(Q.exists(W.countryCapital(x0))(x2 => Q.===(x2, W.cityId(x1))))(
            Q.where(Q.>(W.cityPopulation(x1), Q.int(8000000)))(
                Q.select(Q.tuple2(W.cityName(x1), W.countryName(x0)))
            )
        )
    }
}"""

In [15]:
largeCapitalsNormalized(new NormalizedQUEΛ[StringRep], new NormalizedWorldModel[StringRep]).dyn.apply("")(0)

res14: String = """Q.from(
    W.allCountries
){ x0 => 
    Q.from(
        W.allCities
    ){ x1 => 
        Q.where(Q.&&(Q.exists(W.countryCapital(x0))(x2 => Q.===(x2, W.cityId(x1))), Q.>(W.cityPopulation(x1), Q.int(8000000))))(
            Q.select(Q.tuple2(W.cityName(x1), W.countryName(x0)))
        )
    }
}"""

In [16]:
largeCapitals(new NormalizedQUEΛ[StringRep], new NormalizedWorldModel[StringRep]).dyn.apply("")(0)

res15: String = """Q.from(
    W.allCountries
){ x0 => 
    Q.from(
        W.allCities
    ){ x1 => 
        Q.where(Q.&&(Q.exists(W.countryCapital(x0))(x2 => Q.===(x2, W.cityId(x1))), Q.>(W.cityPopulation(x1), Q.int(8000000))))(
            Q.select(Q.tuple2(W.cityName(x1), W.countryName(x0)))
        )
    }
}"""

### Will you, please, show me the SQL query!?


Ok, let's recall our goal: we want to obtain a simple SQL select statement for `QUEΛ` expressions of type `List`, without nested subqueries in the `FROM` clause. This amounts to somehow implementing a `type SQLFragment[T]=Fragment` interpreter for QUEΛ, but, not for unrestricted expressions, but normalized ones. So, we will proceed as follows: first, we give an interpreter of QUEΛ and WorldModel for `SQLFragment`, but only for those constructs which are normalized. The generation of SQL fragments for comprehension expressions will be given for `Normalized` expressions. Let's see first the former interpreters:

In [17]:
case class SQLFragment[T](fr: Fragment)

object SQLFragment{
    
    implicit object FragmentQUEΛ extends QUEΛ[SQLFragment]{
        def bool(b: Boolean) = SQLFragment(Fragment.const0(b.toString))
        def int(i: Int) = SQLFragment(Fragment.const0(i.toString))
        def str(s: String) = SQLFragment(Fragment.const0(s))

        def ===(a1: SQLFragment[Int], a2: SQLFragment[Int]) = 
            SQLFragment(a1.fr ++ Fragment.const0(" = ") ++ a2.fr)
        
        def >(i1: SQLFragment[Int], i2: SQLFragment[Int]) = 
            SQLFragment(i1.fr ++ Fragment.const0(" > ") ++ i2.fr)
        
        def &&(i1: SQLFragment[Boolean], i2: SQLFragment[Boolean]) = 
            SQLFragment(i1.fr ++ Fragment.const0(" and ") ++ i2.fr)

        def tuple2[A, B](a: SQLFragment[A], b: SQLFragment[B]) = 
            SQLFragment(a.fr ++ fr0", " ++ b.fr)

        def none[A] = SQLFragment(fr0"")
        def some[A](a: SQLFragment[A]) = SQLFragment(a.fr)
        def exists[A](o: SQLFragment[Option[A]])(cond: SQLFragment[A] => SQLFragment[Boolean]): SQLFragment[Boolean] = 
            cond(SQLFragment(o.fr))

        def from[A, B](q: SQLFragment[List[A]])(f: SQLFragment[A] => SQLFragment[List[B]]): SQLFragment[List[B]] = 
            SQLFragment(fr0"won't be used")
        def where[A](cond: SQLFragment[Boolean])(q: SQLFragment[List[A]]): SQLFragment[List[A]] = 
            SQLFragment(fr0"won't be used")
        def select[A](a: SQLFragment[A]): SQLFragment[List[A]] = 
            SQLFragment(fr0"won't be used")
    }
    
    implicit object FragmentWorldModel extends WorldModel[SQLFragment]{

        private def prj(_var: Fragment, column: String): Fragment = 
            _var ++ Fragment.const0("." + column)

        private def table(name: String): Fragment = 
            Fragment.const0(name)

        // Cities

        def cityId(city: SQLFragment[City]) =
            SQLFragment(prj(city.fr, "id"))

        def cityName(city: SQLFragment[City]) =
            SQLFragment(prj(city.fr, "name"))

        def cityCountry(city: SQLFragment[City]) =
            SQLFragment(prj(city.fr, "country"))

        def cityPopulation(city: SQLFragment[City]) = 
            SQLFragment(prj(city.fr, "population"))

        // Countries

        def countryCode(country: SQLFragment[Country]) =
            SQLFragment(prj(country.fr, "code"))

        def countryName(country: SQLFragment[Country]) =
            SQLFragment(prj(country.fr, "name"))

        def countryCapital(country: SQLFragment[Country]) =
            SQLFragment(prj(country.fr, "capital"))

        // World

        def allCountries = SQLFragment(table("country"))

        def allCities = SQLFragment(table("city"))
    }
}

defined class SQLFragment
defined object SQLFragment

The `FragmentWorldModel` instance defines the functional-relational mapping between the tagless-final specification and the relational schema of the World database. Now we are ready to create an interpreter (GADT-based) for normalized queries which built upon the previous tagless-final interpreter:

In [18]:
implicit class FragmentNormalized[A](expr: Normalized[SQLFragment, A]){
    
    def fragment: Fragment = expr match {
        case e: Unk[SQLFragment, _] => 
            e.rep.fr
        case e: Table[SQLFragment, _] => 
            fr"select * from" ++ e.rep.fr
        case e: Single[SQLFragment, _] if e.cond.isEmpty => 
            fr"select" ++ e.returns.fr
        case e: Single[SQLFragment, _] => 
            fr"select" ++ e.returns.fr ++ fr" where" ++ e.cond.map(_.fr).reduce(_ ++ _)
        case e: Compr[SQLFragment, _, _] => {
            val (select, from, where) = comprFragment(e, 0, Nil)
            fr"select" ++ select ++ fr" from" ++ from ++ fr" where" ++ where
        }
    }
    
    private def comprFragment[B, C](d: Compr[SQLFragment, B, C], 
                                    counter: Int, 
                                    from: List[Fragment]): (Fragment, Fragment, Fragment) = {
        val _var = Fragment.const0(s"x$counter")
        val fromEntry = d.table.rep.fr ++ fr"" ++ _var
        
        d.cont(SQLFragment(_var)) match {
            case Unk(rep) => 
                (rep.fr, 
                 (fromEntry :: from).reduce(_ ++ fr"," ++ _), 
                 fr0"")
            case t@Table(rep) =>
                ??? // t.fragment ++ fromEntry
            case Single(where, select) => 
                (select.fr, 
                 (fromEntry :: from).reduce(_ ++ fr"," ++ _), 
                 where.map(_.fr).reduce(_ ++ fr"and" ++ _))
            case c@Compr(_, _) => 
                comprFragment(c, counter + 1, fromEntry :: from)
        }
    }       
}

defined class FragmentNormalized

As we see, the fragment generation for base and ADTs is straightforward. And the fragment corresponding to list representations is a simple SQL select statement with no nested subqueries in the from clause (since this clause is made of atomic _tables_). In order to obtain a full-fledged doobie query, the only thing needed is to summon the `Read` instance for the record type `A`: 

In [19]:
implicit class ListDoobieRepToQuery0[A: Read](d: Normalized[SQLFragment, List[A]]){
    def query: Query0[A] = 
        d.fragment.query[A](Read[A])
}


defined class ListDoobieRepToQuery0

In [20]:
largeCapitals(
    new NormalizedQUEΛ[SQLFragment], 
    new NormalizedWorldModel[SQLFragment]).fragment

res19: Fragment = Fragment("select x1.name, x0.name from city x1, country x0 where x0.capital = x1.id and x1.population > 8000000")

It looks nice! Let's execute it:

In [21]:
largeCapitals(
    new NormalizedQUEΛ[SQLFragment], 
    new NormalizedWorldModel[SQLFragment]).query.to[List].transact(xa).unsafeRunSync

res20: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Seoul", "South Korea"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation")
)

### Does modularity affects the generation?

Let's check the generated SQL query for a more modular implementation:

In [22]:
def city[Repr[_]: QUEΛ: WorldModel](cityId: Repr[Option[Int]]): Repr[List[City]] = 
    for {
        c <- allCities if cityId.exists{ _ === c.id }
    } yield c

def largeCity[Repr[_]: QUEΛ: WorldModel](cityId: Repr[Option[Int]]): Repr[List[City]] = 
    for {
        c <- city(cityId)
        if c.population > 8000000
    } yield c

def largeCapitalsModular[Repr[_]: QUEΛ: WorldModel]: Repr[List[(String, String)]] = 
    for {
        country <- allCountries
        city <- largeCity(country.capital)
    } yield (city.name, country.name)

defined function city
defined function largeCity
defined function largeCapitalsModular

The raw, pre-processed query looks like pretty messy: 

In [23]:
largeCapitalsModular[StringRep].apply("")(0)

res22: String = """Q.from(
    W.allCountries
){ x0 => 
    Q.from(
        Q.from(
            Q.from(
                Q.from(
                    Q.from(
                        W.allCities
                    ){ x1 => 
                        Q.where(Q.exists(W.countryCapital(x0))(x2 => Q.===(x2, W.cityId(x1))))(
                            Q.select(x1)
                        )
                    }
                ){ x1 => 
                    Q.select(x1)
                }
            ){ x1 => 
                Q.where(Q.>(W.cityPopulation(x1), Q.int(8000000)))(
                    Q.select(x1)
                )
            }
        ){ x1 => 
            Q.select(x1)
        }
    ){ x1 => 
        Q.select(Q.tuple2(W.cityName(x1), W.countryName(x0)))
    }
}"""

Still ... 

In [24]:
city(new NormalizedQUEΛ[StringRep].some(new NormalizedQUEΛ[StringRep].int(0)))(
    new NormalizedQUEΛ[StringRep], 
    new NormalizedWorldModel[StringRep]).dyn.apply("")(0)
// == largeCapitals(DoobieRepQUEΛ, DoobieRepWorldModel).fragment.toString

res23: String = """Q.from(
    W.allCities
){ x0 => 
    Q.where(Q.exists(some(Q.int(0)))(x1 => Q.===(x1, W.cityId(x0))))(
        Q.select(x0)
    )
}"""

In [25]:
largeCapitalsModular(
    new NormalizedQUEΛ[StringRep], 
    new NormalizedWorldModel[StringRep]).dyn.apply("")(0)
// == largeCapitals(DoobieRepQUEΛ, DoobieRepWorldModel).fragment.toString

res24: String = """Q.from(
    W.allCountries
){ x0 => 
    Q.from(
        W.allCities
    ){ x1 => 
        Q.where(Q.&&(Q.exists(W.countryCapital(x0))(x2 => Q.===(x2, W.cityId(x1))), Q.>(W.cityPopulation(x1), Q.int(8000000))))(
            Q.select(Q.tuple2(W.cityName(x1), W.countryName(x0)))
        )
    }
}"""

In [26]:
largeCapitalsModular(
    new NormalizedQUEΛ[SQLFragment], 
    new NormalizedWorldModel[SQLFragment]).fragment.toString 
// == largeCapitals(DoobieRepQUEΛ, DoobieRepWorldModel).fragment.toString

res25: String = "Fragment(\"select x1.name, x0.name from city x1, country x0 where x0.capital = x1.id and x1.population > 8000000\")"

Our semantics generate the same exact same query!